# How to generate 1 output TIFF per acquisition in a specified AOI and time range

## Imports

In [ ]:
from sentinelhub import (
    SHConfig,
    BBox,
    bbox_to_dimensions,
    CRS,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    MimeType,
    MosaickingOrder
)

Authentification with config class ([instructions on the configuration](https://sentinelhub-py.readthedocs.io/en/latest/configure.html)).

In [2]:
config = SHConfig()

## Catalog API
Initiate Catalog API instance with config class.

In [3]:
catalog = SentinelHubCatalog(config=config)

Specify collection, AOI and output size.

In [4]:
collection = DataCollection.SENTINEL2_L2A
testing_bbox = BBox(
    [12.44693, 41.870072, 12.541001, 41.917096],
    crs=CRS.WGS84)
time_interval = "2023-01-01", "2023-02-01"
resolution = 10
bbox_size = bbox_to_dimensions(testing_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {bbox_size} pixels")

Image shape at 10 m resolution: (796, 499) pixels


Search catalog entries for specified AOI and time range to save the scene and acquisition dates.

In [5]:
search_iterator = catalog.search(
    collection,
    bbox=testing_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

In [6]:
results = list(search_iterator)
results

[{'id': 'S2A_MSIL2A_20230130T100241_N0509_R122_T32TQM_20230130T142458',
  'properties': {'datetime': '2023-01-30T10:09:10Z'}},
 {'id': 'S2A_MSIL2A_20230130T100241_N0509_R122_T33TTG_20230130T142458',
  'properties': {'datetime': '2023-01-30T10:09:10Z'}},
 {'id': 'S2B_MSIL2A_20230125T100209_N0509_R122_T32TQM_20230125T112820',
  'properties': {'datetime': '2023-01-25T10:09:09Z'}},
 {'id': 'S2B_MSIL2A_20230125T100209_N0509_R122_T33TTG_20230125T112820',
  'properties': {'datetime': '2023-01-25T10:09:09Z'}},
 {'id': 'S2A_MSIL2A_20230120T100331_N0509_R122_T32TQM_20230120T142209',
  'properties': {'datetime': '2023-01-20T10:09:06Z'}},
 {'id': 'S2A_MSIL2A_20230120T100331_N0509_R122_T33TTG_20230120T142209',
  'properties': {'datetime': '2023-01-20T10:09:06Z'}},
 {'id': 'S2B_MSIL2A_20230115T100249_N0509_R122_T32TQM_20230115T112323',
  'properties': {'datetime': '2023-01-15T10:09:08Z'}},
 {'id': 'S2B_MSIL2A_20230115T100249_N0509_R122_T33TTG_20230115T112323',
  'properties': {'datetime': '2023-01-1

Create unique timestamps from the Catalog acquisition list.

In [7]:
timestamps = sorted(set(tile['properties']['datetime'].split('T')[0].replace("-", "") for tile in results))
timestamps

['20230105', '20230110', '20230115', '20230120', '20230125', '20230130']

Define the output dictionaries (string) we want to pass into the evalscript's `function setup()`.

In [8]:
output_list = [f'{{id: "acq_{scene}", bands: 2}}' for scene in timestamps]
output_str = ", ".join(output_list)
output_str

'{id: "acq_20230105", bands: 2}, {id: "acq_20230110", bands: 2}, {id: "acq_20230115", bands: 2}, {id: "acq_20230120", bands: 2}, {id: "acq_20230125", bands: 2}, {id: "acq_20230130", bands: 2}'

Define the return dictionary (string) we want to pass into the evalscript's `function evaluatePixel()`.

In [9]:
return_list = [f'acq_{scene}: [b03_array[{index}], b04_array[{index}]]' for index, scene in enumerate(timestamps)]
return_str = ", ".join(return_list)
return_str

'acq_20230105: [b03_array[0], b04_array[0]], acq_20230110: [b03_array[1], b04_array[1]], acq_20230115: [b03_array[2], b04_array[2]], acq_20230120: [b03_array[3], b04_array[3]], acq_20230125: [b03_array[4], b04_array[4]], acq_20230130: [b03_array[5], b04_array[5]]'

## Process API

Define the evalscript for the requests. Please note that we have to use double curly brackets throughout the script because we are inserting external variables with the output and return strings we defined above.

In [10]:
evalscript = f"""
//VERSION=3
function setup() {{
  return {{
    input: ["B03", "B04"],
    output: [{output_str}],
    mosaicking: "ORBIT"
  }};
}}

function evaluatePixel(samples) {{
  // Precompute an array to contain observations
  var n_observations = samples.length;
  let b03_array = new Array(n_observations).fill(NaN);
  let b04_array = new Array(n_observations).fill(NaN);
  
  // Fill the array with values
  samples.forEach((sample, index) => {{
    b03_array[index] = sample.B03;
    b04_array[index] = sample.B04;
  }});

  //return band_array
  return {{{return_str}}};
}}
"""

In [11]:
evalscript

'\n//VERSION=3\nfunction setup() {\n  return {\n    input: ["B03", "B04"],\n    output: [{id: "acq_20230105", bands: 2}, {id: "acq_20230110", bands: 2}, {id: "acq_20230115", bands: 2}, {id: "acq_20230120", bands: 2}, {id: "acq_20230125", bands: 2}, {id: "acq_20230130", bands: 2}],\n    mosaicking: "ORBIT"\n  };\n}\n\nfunction evaluatePixel(samples) {\n  // Precompute an array to contain observations\n  var n_observations = samples.length;\n  let b03_array = new Array(n_observations).fill(NaN);\n  let b04_array = new Array(n_observations).fill(NaN);\n  \n  // Fill the array with values\n  samples.forEach((sample, index) => {\n    b03_array[index] = sample.B03;\n    b04_array[index] = sample.B04;\n  });\n\n  //return band_array\n  return {acq_20230105: [b03_array[0], b04_array[0]], acq_20230110: [b03_array[1], b04_array[1]], acq_20230115: [b03_array[2], b04_array[2]], acq_20230120: [b03_array[3], b04_array[3]], acq_20230125: [b03_array[4], b04_array[4]], acq_20230130: [b03_array[5], b04_

Define the request responses with the same identifiers as in the evalscript.

In [12]:
response_list = [SentinelHubRequest.output_response(f"acq_{scene}", MimeType.TIFF) for scene in timestamps]
response_list

[{'identifier': 'acq_20230105', 'format': {'type': 'image/tiff'}},
 {'identifier': 'acq_20230110', 'format': {'type': 'image/tiff'}},
 {'identifier': 'acq_20230115', 'format': {'type': 'image/tiff'}},
 {'identifier': 'acq_20230120', 'format': {'type': 'image/tiff'}},
 {'identifier': 'acq_20230125', 'format': {'type': 'image/tiff'}},
 {'identifier': 'acq_20230130', 'format': {'type': 'image/tiff'}}]

Prepare the request payload.

In [13]:
request = SentinelHubRequest(
    data_folder="test_dir",
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=collection,
            time_interval=time_interval,
            mosaicking_order=MosaickingOrder.LEAST_RECENT
        )
    ],
    responses=response_list,
    bbox=testing_bbox,
    size=bbox_size,
    config=config,
)

In [14]:
request.get_data(save_data=True)

[{'acq_20230105.tif': array([[[ 41,  38],
          [ 48,  46],
          [ 54,  52],
          ...,
          [ 83,  84],
          [ 84,  84],
          [ 82,  82]],
  
         [[ 43,  38],
          [ 52,  51],
          [ 56,  57],
          ...,
          [ 81,  81],
          [ 81,  78],
          [ 81,  78]],
  
         [[ 44,  40],
          [ 56,  53],
          [ 58,  58],
          ...,
          [ 76,  77],
          [ 75,  74],
          [ 79,  80]],
  
         ...,
  
         [[125, 127],
          [119, 120],
          [114, 114],
          ...,
          [ 27,  27],
          [ 27,  26],
          [ 32,  28]],
  
         [[128, 132],
          [121, 127],
          [121, 127],
          ...,
          [ 30,  30],
          [ 28,  31],
          [ 31,  33]],
  
         [[139, 143],
          [135, 139],
          [134, 138],
          ...,
          [ 31,  30],
          [ 35,  35],
          [ 36,  33]]], dtype=uint8),
  'acq_20230110.tif': array([[[10,  5],
     